In [1]:
''' GAN 모델 구성하기 '''

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

# 실제 이미지는 28 * 28 = 784개의 특징을 가집니다.
X = tf.placeholder(tf.float32, [None, 784])

# 가짜 이미지를 위해 필요한 노이즈의 크기는 128입니다.
Z = tf.placeholder(tf.float32, [None, 128])

# 생성자: 128(노이즈) → 256(은닉층) → 784(입력층)
G_W1 = tf.Variable(tf.random_normal([128, 256], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([256]))
G_W2 = tf.Variable(tf.random_normal([256, 784], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([784]))

# 구분자: 784(입력층) → 256(은닉층) → 0 ~ 1(일치도)
D_W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([256]))
D_W2 = tf.Variable(tf.random_normal([256, 1], stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([1]))

# 생성자 객체를 생성하는 함수입니다.
def generator(noise):
    hidden = tf.nn.relu(tf.matmul(noise, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

# 구분자 객체를 생성하는 함수입니다.
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

# 무작위 노이즈를 생성합니다.
def get_noise(batch_size, noise):
    return np.random.normal(size=(batch_size, noise))

# 가짜 이미지 생성자는 128 크기의 노이즈에서 불러옵니다.
G = generator(Z)
# 가짜 이미지 구분자는 128 크기의 노이즈가 생성한 784 크기의 이미지에서 불러옵니다.
D_gene = discriminator(G)
# 실제 이미지 구분자는 784 크기의 이미지에서 불러옵니다.
D_real = discriminator(X)

# 구분자의 손실 함수는 진짜 이미지가 1에 가깝고, 가짜 이미지가 0에 가깝도록 설정합니다.
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
# 생성자의 손실 함수는 가짜 이미지가 1에 가깝도록 설정합니다.
loss_G = tf.reduce_mean(tf.log(D_gene))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
''' GAN 모델 학습 준비하기 '''

# 구분자는 구분자 가중치 및 바이어스만을 사용합니다.
D_var_list = [D_W1, D_b1, D_W2, D_b2]
# 생성자는 생성자 가중치 및 바이어스만을 사용합니다.
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# 구분자 최적화를 진행합니다.
train_D = tf.train.AdamOptimizer(0.001).minimize(-loss_D, var_list=D_var_list)
# 생성자 최적화를 진행합니다.
train_G = tf.train.AdamOptimizer(0.001).minimize(-loss_G, var_list=G_var_list)

# 세션을 생성해 그래프를 동작시킵니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 구분자와 생성자의 비용 변수를 생성합니다.
loss_val_D, loss_val_G = 0, 0

# 배치 크기를 100으로 설정합니다.
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [3]:
''' GAN 모델 학습 수행 및 이미지 결과 확인하기 '''

# 총 100번 학습을 진행합니다.
for epoch in range(100):
    # 전체 배치 크기만큼 반복합니다.
    for i in range(total_batch):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, 128)
        # 구분자는 실제 이미지 및 노이즈를 이용해 학습을 진행합니다.
        _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_x, Z: noise})
        # 생성자는 노이즈만을 이용해 학습을 진행합니다.
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
    # 1번 돌 때마다 학습 상황을 출력합니다.
    print('학습:', '%04d' % epoch,
          '구분자 오차: {:.4}'.format(loss_val_D),
          '생성자 오차: {:.4}'.format(loss_val_G))
    
    # 10번 돌 때마다 결과를 그림으로 확인합니다.
    if epoch == 0 or (epoch + 1) % 10 == 0:
        # 샘플 이미지의 크기는 10입니다.
        size = 10
        noise = get_noise(size, 128)

        # 생성자가 임의의 샘플 이미지를 생성하도록 합니다.
        samples = sess.run(G, feed_dict={Z: noise})
        
        # 만든 그림을 폴더에 출력할 수 있도록 합니다.
        fig, ax = plt.subplots(1, size, figsize=(size, 1))
        for i in range(size):
            ax[i].set_axis_off()
            # 28 * 28 크기의 이미지를 생성합니다.
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(4)), bbox_inches='tight')
        plt.close(fig)

학습: 0000 구분자 오차: -0.3718 생성자 오차: -3.343
학습: 0001 구분자 오차: -0.1736 생성자 오차: -5.947
학습: 0002 구분자 오차: -0.08865 생성자 오차: -5.5
학습: 0003 구분자 오차: -0.6135 생성자 오차: -5.729
학습: 0004 구분자 오차: -0.1609 생성자 오차: -5.075
학습: 0005 구분자 오차: -0.3523 생성자 오차: -3.03
학습: 0006 구분자 오차: -0.4022 생성자 오차: -4.839
학습: 0007 구분자 오차: -1.069 생성자 오차: -2.47
학습: 0008 구분자 오차: -0.7203 생성자 오차: -2.873
학습: 0009 구분자 오차: -0.826 생성자 오차: -2.835
학습: 0010 구분자 오차: -1.276 생성자 오차: -2.782
학습: 0011 구분자 오차: -0.631 생성자 오차: -3.139
학습: 0012 구분자 오차: -1.496 생성자 오차: -1.578
학습: 0013 구분자 오차: -0.9282 생성자 오차: -2.917
학습: 0014 구분자 오차: -0.8063 생성자 오차: -3.007
학습: 0015 구분자 오차: -0.9619 생성자 오차: -2.891
학습: 0016 구분자 오차: -0.9439 생성자 오차: -2.299
학습: 0017 구분자 오차: -0.7772 생성자 오차: -2.89
학습: 0018 구분자 오차: -0.8019 생성자 오차: -2.157
학습: 0019 구분자 오차: -0.9196 생성자 오차: -2.109
학습: 0020 구분자 오차: -0.6557 생성자 오차: -2.521
학습: 0021 구분자 오차: -0.7813 생성자 오차: -2.575
학습: 0022 구분자 오차: -0.9912 생성자 오차: -2.173
학습: 0023 구분자 오차: -0.7456 생성자 오차: -2.4
학습: 0024 구분자 오차: -0.5747 생성자 오차: -2.734
학습: 0025 구분